In [15]:
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
from collections import defaultdict
import re

from src.datasets.who_is_who import WhoIsWhoDataset
from src.shared.database_wrapper import DatabaseWrapper
from src.shared.graph_schema import NodeType, EdgeType
from src.shared.graph_sampling import GraphSampling

## Configuration

In [16]:
max_iterations = 20000
add_true_authors = False
link_true_authors = True
link_co_authors = True
link_title = True
link_abstract = True
link_venue = True

co_author_overlap_threshold = 0.15
link_title_threshold = 0.7
link_abstract_threshold = 0.7
link_venue_threshold = 0.7

link_title_k = 15
link_abstract_k = 15
link_venue_k = 8

In [17]:

db = DatabaseWrapper(database='dense-graph')
db.delete_all_nodes()

2024-11-25 09:37:57,059 - DatabaseWrapper - INFO - Connecting to the database ...
2024-11-25 09:37:57,060 - DatabaseWrapper - INFO - Database ready.
2024-11-25 09:37:57,138 - DatabaseWrapper - INFO - Deleted all nodes.


In [18]:
data = WhoIsWhoDataset.parse_data()
train_data = WhoIsWhoDataset.parse_train()

In [19]:
model = SentenceTransformer(
    'sentence-transformers/all-MiniLM-L6-v2',
    device='cuda'
)
print(f"Model dim: {model.get_sentence_embedding_dimension()}")

Model dim: 384


## Add publication nodes to the graph database

In [20]:
def process_batch(batch):
    if not batch[NodeType.PUBLICATION]:
        return
    title_embs = model.encode(
        [node['title'] for node in batch[NodeType.PUBLICATION]]
    )
    abstract_embs = model.encode(
        [node['abstract'] for node in batch[NodeType.PUBLICATION]]
    )
    venue_embs = model.encode(
        [node['venue'] for node in batch[NodeType.PUBLICATION]]
    )
    for i, node in enumerate(batch[NodeType.PUBLICATION]):
        node['title_emb'] = title_embs[i]
        node['abstract_emb'] = abstract_embs[i]
        node['venue_emb'] = venue_embs[i]
        # vertically stack the embeddings
        node['feature_vec'] = list(title_embs[i]) + list(abstract_embs[i])
    db.merge_nodes(NodeType.PUBLICATION, batch[NodeType.PUBLICATION])
    batch[NodeType.PUBLICATION] = []

In [21]:
batch_nodes = defaultdict(list)
current_iteration = 0

with tqdm(total=max_iterations) as pbar:
    for author_id, values in train_data.items():
        papers = values.get('normal_data', [])
        papers.extend(values.get('outliers', []))
        
        if max_iterations is not None and current_iteration >= max_iterations:
            break
        current_iteration += len(papers)
        
        for paper_id in papers:
            values = data[paper_id]
            paper_node = {
                'id': values['id'],
                'title': values['title'],
                'abstract': values['abstract'],
                'year': values['year'],
                'venue': values['venue'],
            }
            batch_nodes[NodeType.PUBLICATION].append(paper_node)
            
            if len(batch_nodes[NodeType.PUBLICATION]) % 1000 == 0:
                process_batch(batch_nodes)
                
            pbar.update(1)
        
process_batch(batch_nodes)

  0%|          | 0/20000 [00:00<?, ?it/s]

/home/vincie/.anaconda3/envs/master/lib/python3.9/site-packages/neo4j/_sync/work/result.py:500: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "


## Add true author data to the nodes

In [22]:
def reverse_dict(author_dict):
    paper_to_author = {}
    for author_id, values in author_dict.items():
        normal_papers = values.get('normal_data', [])
        for paper_id in normal_papers:
            paper_to_author[paper_id] = author_id
    return paper_to_author

def add_true_authors(db: DatabaseWrapper, train_data):
    paper_id_to_author = reverse_dict(train_data)

    with tqdm(total=db.count_nodes(NodeType.PUBLICATION), desc="Merging WhoIsWho train_author.json") as pbar:
        for nodes in db.iter_nodes(NodeType.PUBLICATION, ['id']):
            for node in nodes:
                true_author_id = paper_id_to_author.get(node['id'], '')
                true_author_name = train_data.get(true_author_id, {}).get('name', '')
                
                db.merge_properties(
                    type=NodeType.PUBLICATION, 
                    node_id=node['id'], 
                    properties={'true_author_id': true_author_id, 'true_author_name': true_author_name}
                )
                pbar.update(1)

if add_true_authors:
    add_true_authors(db, train_data)

Merging WhoIsWho train_author.json:   0%|          | 0/20034 [00:00<?, ?it/s]

## Link publication nodes if they share the same true author

In [23]:
import random

def get_sample_size(num_papers):
    random_k = 3 + random.random() * 0.2
    return int(num_papers**0.5 * random_k)

def link_true_authors(db: DatabaseWrapper, train_data):
    node_type = NodeType.PUBLICATION
    edges_to_merge = []
    current_iteration = 0
    
    with tqdm(total=max_iterations, desc="Merging true-author edges") as pbar:
        for author_id, values in train_data.items():
            papers = values.get('normal_data', [])
            print(f"Number of papers: {len(papers)}")
            
            pbar.update(len(papers) + len(values.get('outliers', [])))
            if max_iterations is not None and current_iteration >= max_iterations:
                break 
            current_iteration += len(papers) + len(values.get('outliers', []))
            
            for i in range(len(papers)):
                for j in range(len(papers)):
                    if i == j:
                        continue
                    edges_to_merge.append([papers[i], papers[j]])
            
            print(f"Number of edges to merge: {len(edges_to_merge)}")
            # Randomly sample 10-40% of the edges_to_merge list
            sample_size = get_sample_size(len(edges_to_merge))
            print(f"Sample size: {sample_size}")
            edges_to_merge = random.sample(edges_to_merge, sample_size)
            print(f"Sampled number of edges to merge: {len(edges_to_merge)}")
             
            if len(edges_to_merge) > 1000:
                db.merge_edges(start_label=node_type, end_label=node_type, edge_type=EdgeType.SAME_AUTHOR, edges=edges_to_merge)
                edges_to_merge.clear()
            
            
            
        
        if edges_to_merge:
            db.merge_edges(start_label=node_type, end_label=node_type, edge_type=EdgeType.SAME_AUTHOR, edges=edges_to_merge)
            edges_to_merge.clear()
            
if link_true_authors:
    db.delete_edges(EdgeType.SAME_AUTHOR)
    link_true_authors(db, train_data)

Merging true-author edges:   0%|          | 0/20000 [00:00<?, ?it/s]

Number of papers: 565
Number of edges to merge: 318660
Sample size: 1731
Sampled number of edges to merge: 1731
Number of papers: 40
Number of edges to merge: 1560
Sample size: 119
Sampled number of edges to merge: 119
Number of papers: 74
Number of edges to merge: 5521
Sample size: 233
Sampled number of edges to merge: 233
Number of papers: 53
Number of edges to merge: 2989
Sample size: 167
Sampled number of edges to merge: 167
Number of papers: 36
Number of edges to merge: 1427
Sample size: 116
Sampled number of edges to merge: 116
Number of papers: 34
Number of edges to merge: 1238
Sample size: 105
Sampled number of edges to merge: 105
Number of papers: 149
Number of edges to merge: 22157
Sample size: 457
Sampled number of edges to merge: 457
Number of papers: 145
Number of edges to merge: 21337
Sample size: 439
Sampled number of edges to merge: 439
Number of papers: 36
Number of edges to merge: 1699
Sample size: 131
Sampled number of edges to merge: 131
Number of papers: 172
Number

## Link nodes based on co-author relationships

In [24]:
def link_co_author_network(db: DatabaseWrapper, node_type: NodeType):
    num_nodes = db.count_nodes(node_type)
    attrs = ['id']
    co_author_map = defaultdict(list)
    co_author_overlap = defaultdict(int)
        
    print(f"Linking {node_type.value} nodes based on co-authorship ...")
    with tqdm(total=num_nodes, desc=f"Progress {node_type.value} co-authorship") as pbar:
        for nodes in db.iter_nodes(node_type, attrs):
            for node in nodes:
                co_authors = [author["name"] for author in data[node['id']]['authors']]
                for author in co_authors:
                    name = author.strip()
                    name = re.sub(r'[^A-Za-z\s]', '', name)
                    name_parts = name.split()
                    if len(name_parts) == 0:
                        continue
                    
                    surname = name_parts[-1]
                    given_name_initial = (name_parts[0] if len(name_parts) > 1 else ' ')[0]
                    abbrev = f"{surname} {given_name_initial}"
                    co_author_map[abbrev].append(node['id'])
                pbar.update(1)
    
    for k, v in co_author_map.items():
        for i in range(len(v)):
            for j in range(i + 1, len(v)):
                co_author_overlap[(v[i], v[j])] += 1
                
    for k, v in co_author_overlap.items():
        total_num_authors = data[k[0]]["authors"] + data[k[1]]["authors"]
        co_author_overlap[k] = v / len(total_num_authors)
    
    print(f"Max. co-authors: {max(len(v) for v in co_author_map.values())}")
    print(f"Max. co-author overlap: {max(co_author_overlap.values())}")
    
    print("Number of co-author pairs:", len(co_author_overlap))
    print("Number of co-author pairs with overlap > 0.25:", len([v for v in co_author_overlap.values() if v > co_author_overlap_threshold]))
    
    print("Merging edges ...")
    edges_to_merge = [[k[0], k[1], {'sim': v}] for k, v in co_author_overlap.items() if v > co_author_overlap_threshold]
    with tqdm(total=len(edges_to_merge), desc="Merging co-author edges") as pbar:
        for i in range(0, len(edges_to_merge), 1000):
            db.merge_edges_with_properties(start_label=node_type, end_label=node_type, edge_type=EdgeType.SIM_AUTHOR, edges=edges_to_merge[i:i+1000])
            pbar.update(1000)

In [25]:
if link_co_authors:
    link_co_author_network(db, NodeType.PUBLICATION)

Linking Publication nodes based on co-authorship ...


Progress Publication co-authorship:   0%|          | 0/20034 [00:00<?, ?it/s]

Max. co-authors: 1796
Max. co-author overlap: 1.2181571815718157
Number of co-author pairs: 6293688
Number of co-author pairs with overlap > 0.25: 776093
Merging edges ...


Merging co-author edges:   0%|          | 0/776093 [00:00<?, ?it/s]

## Link nodes based on cosine similarity of their embeddings

In [28]:
def link_node_attr_cosine(db: DatabaseWrapper, node_type: NodeType, vec_attr: str, edge_type: EdgeType, threshold: float = 0.7, k=8, filter_empty_original_attr: str = None):
    num_nodes = db.count_nodes(node_type)
    edges = []
    attrs = ['id', vec_attr]
    if filter_empty_original_attr:
        attrs.append(filter_empty_original_attr)
        
    print(f"Linking {node_type.value} nodes based on {vec_attr} attribute ...")
    with tqdm(total=num_nodes, desc=f"Progress {node_type.value} {vec_attr}") as pbar:
        for nodes in db.iter_nodes(node_type, attrs):
            for node in nodes:
                if filter_empty_original_attr and not node[filter_empty_original_attr]:
                    pbar.update(1)
                    print(f"Skipping node {node['id']} because {filter_empty_original_attr} is empty")
                    continue
                similar_nodes = db.get_similar_nodes_vec(
                    node_type,
                    vec_attr,
                    node[vec_attr],
                    threshold,
                    k
                )
                for ix, row in similar_nodes.iterrows():
                    if row['id'] == node['id']:
                        continue
                    edges.append([node['id'], row['id'], {"sim": row['sim']}])
                    #db.merge_edge(node_type, node['id'], node_type, row['id'], edge_type, {"sim": row['sim']})
                if len(edges) > 1000:
                    print(f"Merging {len(edges)} edges ...")
                    #db.merge_edges(start_label=node_type, end_label=node_type, edge_type=edge_type, edges=edges)
                    db.merge_edges_with_properties(start_label=node_type, end_label=node_type, edge_type=edge_type, edges=edges)
                    edges.clear()
                    
                pbar.update(1)
    if edges:
        db.merge_edges(start_label=node_type, end_label=node_type, edge_type=edge_type, edges=edges)

In [29]:
model_dim = model.get_sentence_embedding_dimension()

# Create vector index and link nodes based on cosine similarity
if link_title:
    db.create_vector_index('title_index', NodeType.PUBLICATION, 'title_emb', model_dim)
    link_node_attr_cosine(db, NodeType.PUBLICATION, 'title_emb', EdgeType.SIM_TITLE, threshold=link_title_threshold, k=link_title_k)
    
if link_abstract:
    db.create_vector_index('abstract_index', NodeType.PUBLICATION, 'abstract_emb', model_dim)
    link_node_attr_cosine(db, NodeType.PUBLICATION, 'abstract_emb', EdgeType.SIM_ABSTRACT, threshold=link_abstract_threshold, k=link_abstract_k)
    
if link_venue:
    db.create_vector_index('venue_index', NodeType.PUBLICATION, 'venue_emb', model_dim)
    link_node_attr_cosine(db, NodeType.PUBLICATION, 'venue_emb', EdgeType.SIM_VENUE, threshold=link_venue_threshold, k=link_venue_k)

Linking Publication nodes based on title_emb attribute ...


Progress Publication title_emb:   0%|          | 0/20034 [00:00<?, ?it/s]

Merging 1004 edges ...
Merging 1001 edges ...
Merging 1002 edges ...
Merging 1003 edges ...
Merging 1005 edges ...
Merging 1002 edges ...
Merging 1007 edges ...
Merging 1002 edges ...
Merging 1007 edges ...
Merging 1001 edges ...
Merging 1001 edges ...
Merging 1003 edges ...
Merging 1004 edges ...
Merging 1004 edges ...
Merging 1001 edges ...
Merging 1003 edges ...
Merging 1005 edges ...
Merging 1005 edges ...
Merging 1013 edges ...
Merging 1001 edges ...
Merging 1009 edges ...
Merging 1002 edges ...
Merging 1001 edges ...
Merging 1009 edges ...
Merging 1014 edges ...
Merging 1005 edges ...
Merging 1004 edges ...
Merging 1001 edges ...
Merging 1003 edges ...
Merging 1003 edges ...
Merging 1004 edges ...
Merging 1015 edges ...
Merging 1001 edges ...
Merging 1001 edges ...
Merging 1006 edges ...
Merging 1004 edges ...
Merging 1001 edges ...
Merging 1004 edges ...
Merging 1006 edges ...
Merging 1009 edges ...
Merging 1007 edges ...
Merging 1001 edges ...
Merging 1007 edges ...
Merging 100

Progress Publication abstract_emb:   0%|          | 0/20034 [00:00<?, ?it/s]

Merging 1012 edges ...
Merging 1002 edges ...
Merging 1004 edges ...
Merging 1010 edges ...
Merging 1013 edges ...
Merging 1005 edges ...
Merging 1012 edges ...
Merging 1001 edges ...
Merging 1014 edges ...
Merging 1008 edges ...
Merging 1011 edges ...
Merging 1001 edges ...
Merging 1008 edges ...
Merging 1005 edges ...
Merging 1007 edges ...
Merging 1013 edges ...
Merging 1002 edges ...
Merging 1003 edges ...
Merging 1013 edges ...
Merging 1003 edges ...
Merging 1013 edges ...
Merging 1009 edges ...
Merging 1007 edges ...
Merging 1001 edges ...
Merging 1012 edges ...
Merging 1001 edges ...
Merging 1003 edges ...
Merging 1002 edges ...
Merging 1009 edges ...
Merging 1002 edges ...
Merging 1002 edges ...
Merging 1006 edges ...
Merging 1001 edges ...
Merging 1010 edges ...
Merging 1009 edges ...
Merging 1003 edges ...
Merging 1004 edges ...
Merging 1004 edges ...
Merging 1001 edges ...
Merging 1004 edges ...
Merging 1001 edges ...
Merging 1010 edges ...
Merging 1009 edges ...
Merging 101

Progress Publication venue_emb:   0%|          | 0/20034 [00:00<?, ?it/s]

Merging 1005 edges ...
Merging 1005 edges ...
Merging 1002 edges ...
Merging 1001 edges ...
Merging 1006 edges ...
Merging 1003 edges ...
Merging 1002 edges ...
Merging 1003 edges ...
Merging 1007 edges ...
Merging 1001 edges ...
Merging 1004 edges ...
Merging 1001 edges ...
Merging 1005 edges ...
Merging 1001 edges ...
Merging 1002 edges ...
Merging 1006 edges ...
Merging 1004 edges ...
Merging 1001 edges ...
Merging 1004 edges ...
Merging 1001 edges ...
Merging 1007 edges ...
Merging 1005 edges ...
Merging 1006 edges ...
Merging 1003 edges ...
Merging 1006 edges ...
Merging 1005 edges ...
Merging 1008 edges ...
Merging 1001 edges ...
Merging 1001 edges ...
Merging 1005 edges ...
Merging 1006 edges ...
Merging 1005 edges ...
Merging 1003 edges ...
Merging 1006 edges ...
Merging 1006 edges ...
Merging 1004 edges ...
Merging 1007 edges ...
Merging 1003 edges ...
Merging 1003 edges ...
Merging 1005 edges ...
Merging 1004 edges ...
Merging 1002 edges ...
Merging 1007 edges ...
Merging 100

In [30]:
# Remove edges if according attribute is empty
if link_title:
    db.delete_edges_for_empty_attr(NodeType.PUBLICATION, EdgeType.SIM_TITLE, 'title')
    
if link_abstract:
    db.delete_edges_for_empty_attr(NodeType.PUBLICATION, EdgeType.SIM_ABSTRACT, 'abstract')
    
if link_venue:
    db.delete_edges_for_empty_attr(NodeType.PUBLICATION, EdgeType.SIM_VENUE, 'venue')

In [31]:
print("Number of publication nodes:", db.count_nodes(NodeType.PUBLICATION))

Number of publication nodes: 20034
